## Post Validation - Build detection History

In [7]:
import pandas as pd
import os
from pathlib import Path

In [3]:
# Load Validation of Selection Table
main_dir = "/mnt/d/night_recordings_analysis/"

val_df = pd.read_csv(os.path.join(main_dir, "Tropical_Screech-OWl/trsowl_Validation.txt"), sep='\t')

In [18]:
val_df['Eval']

0       0
1       0
2       0
3       0
4       0
       ..
2299    1
2300    1
2301    1
2302    1
2303    1
Name: Eval, Length: 2304, dtype: int64

In [8]:
# Extract Site_ID and SN from file name
def parse_site_sn(filename):
    parts = Path(filename).stem.split('_')  # Remove .wav and split
    try:
        site_id = parts[-2]
        sn = parts[-1].replace("sn", "")
        return pd.Series([site_id, sn])
    except Exception as e:
        print(f"Failed to parse filename: {filename}")
        return pd.Series([None, None])

In [9]:
# Apply parsing
val_df[['Site_ID', 'SN']] = val_df['Begin File'].apply(parse_site_sn)

In [11]:
# Make sure Eval is numeric
val_df['Eval'] = pd.to_numeric(val_df['Eval'], errors='coerce').fillna(0).astype(int)

In [19]:
# Pivot to detection history
detection_history = val_df.groupby(['Site_ID', 'SN'])['Eval'].max().unstack(fill_value=0)

In [24]:
detection_history.head()

SN,1,10,11,12,13,14,15,16,17,18,...,32,33,34,35,4,5,6,7,8,9
Site_ID,,,,,,,,,,,,,,,,,,,,,
ch1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ch10,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,1,0,0,1,1
ch11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ch12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ch13,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [23]:
# Sort columns (optional)
detection_history = detection_history.reindex(sorted(detection_history.columns), axis=1)

In [26]:
# Save
detection_history.to_csv(os.path.join(main_dir, "Tropical_Screech-Owl/detection_history_matrix.csv"))

print("✅ Detection history created and saved.")


✅ Detection history created and saved.


Need to add NA to every empty cell

I have manually removed extra zeros to match the number of surveyed nights per site, but maybe we can extract this data to do this automatically after creating the df right? Moreover I had to sort myself the survey nights and sites in the desired order. Survey nights should be treated as numeric to order them sequentially from 01 up to 35. The same with the values from column Site_ID, although they are composed of a letter and a number the letter choud be organized alphabetically and then use the number and sort it from lower to higher values